# Advanced Data Types
- The official doc recorded about 21 data type categories.

### List of data types
- Numeric types
    - Floating-Point types
- Monetary type
    - Money
- Date/Time types
    - `interval`
- Boolean types
- Enumerated types
- Network address types
    - inet
    - cidr
- Text search type
- UUID
- XML
- JSON
- Arrays
- Composite type
- Range types
    - int4range
    - numrange
    - daterange
- Domain types

## Numeric Types
- The numeric data types is made up 
    - `INT`
    - `SERIAL`
    - `floating point`
    - `arbitrary precision numbers`

### Real, Float, Double Precision
- `float8` is an 8-byte floating point number
- `float4` is a 4-byte floating point number
- `float(n)`: where `n` represents the precision
- `numeric(p,s)`: Where `p` is the precision and `s` is the scale.
- `inf`, and `-inf` that represents infinity and -infinity
- `NaN` that represents 'not a number'

## Monetary types
- This category contains the `money` data type
- The `money` data types allow us to represent money with both the value and the currency.
- By default, `money` uses a scale of `2`.
- To see the currency currently set in your system, you can run `SHOW lc_monetary`
- To change the currency, you can set `lc_monetary`.

```sql
-- List all supported currency by your system.
SELECT collname
FROM pg_collation
ORDER BY collname;

-- Set the currency
-- en_US.UTF-8

SET lc_monetary TO '<value>';
SET lc_monetary TO 'en_PH.utf8';
```

## Date/Time
### Time
These were given for self-study
- `time with time zone` or `timez`
- `time without time zone` or `time`
- `timestamp with time zone` or `timestampz`
- `timestamp without time zone` or `timestamp`

Some constants and functions

- `NOW()`: current timestamp with time zone
- `CURRENT_TIMESTAMP`: current timestamp with time zone

### Date
- Date represent `year`, `month`, and `day`.
- In postgres, we have the constant `CURRENT_DATE` that returns the current date

#### datestyle
- postgresql has the `datestyle` configuration that holds the format of your date.
- To view the current setting, run `SHOW datestyle;`
- The official doc list the values we can set to datestyle
    - ISO, MDY
    - ISO, DMY
    - ISO, YMD
    - SQL DMY
    - SQL MDY
    - Postgres, MDY

## Interval
- It allows you to store and manipulate period of time in years, months, days, hours, minutes, seconds, etc

### Example
When was it 1 day, 5 hours, 30 minutes ago.

```sql
SELECT NOW(),
        NOW() - INTERVAL '1 day 5 hours 30 minutes' ago;
```
- Base on the above query
    - Our interval takes in a string of time period
- We can use the following case-insensitive time periods
    - year(s): `y`
    - month(s): `m`
    - week(s): `w`
    - day(s): `d`
    - hour(s): `h`
    - minute(s): `m`
    - second(s): `s`

```sql
SELECT NOW(),
        NOW() - INTERVAL '1D 5H 30M' ago;

-- Confusion
-- No matter where 'M' is found, it represents the `minutes`
SELECT NOW() - INTERVAL '1M 2W 1D 5H 30M 15S';

-- We have to use `P` and `T`. Where `P` indicate the start of date and `T` indicates
-- the start of time

SELECT NOW() - INTERVAL 'P1M2W1DT5H30M15S';
```

## BOOLEAN TYPES

```sql
-- 4
-- 0
-- -1
-- 4.2
-- '0'
-- '1'
-- '4'
-- true
-- false
-- 'true'
-- 'false'
-- 'ok'
-- 't'
-- 'f'
-- 'n'
-- 'y'
-- 'no'
-- 'yes'
```

## Enumerated type
- Creates a list of values that your data should be bound to.

```sql
-- SYNTAX
CREATE TYPE <type-name> AS ENUM(<values>);

-- Example
CREATE TYPE book_category_enum AS ENUM('programming', 'art', 'history' , 'politics', 'other');

-- Alter enumerated types
ALTER TYPE <type-name> ADD VALUE '<value>';
ALTER TYPE <type-name> RENAME VALUE '<old-value>' TO '<new-value>';
ALTER TYPE <type-name> DROP VALUE '<value'>; -- Doesn't work. Look for a proper way to drop a value

-- View all data types
\dT
-- View a single data type
\dT+ <data-type>
```

## JSON(`JavaScript Object Notation`)
- This data type represents data as `key-value` pairs
- postgresql provides two ways for storing json data `json`, `jsonb`
- Benefits of `jsonb`
    - Maintain unique keys like dictionary in Python
    - White spaces are not preserved
    - It doesn't maintain any order.

```sql
CREATE TABLE my_product (
    id SERIAL PRIMARY KEY,
    info JSON NOT NULL,
    detail JSONB
);
-- JSON uses curly braces just like in Python, but enclosed in single quotes
-- and double quote is used to represent strings inside the json.
INSERT INTO my_product (info)
VALUES ('{"name": "oil", "store": {"store_name": "jim", "store_id": 5}}'),
        ('{"name": "mango", "store": {"store_name": "ali", "store_id": 4}}');
```

#### Select
Postgresql provide two powerful operators `->` and `->>` to help query json data
- `->`: Returns JSON fields as objects
- `->>`: Returns JSON fields as text.

```sql
-- SELECT <column> -> '<key-name>' FROM <table>;
SELECT info -> 'name' FROM my_product;
SELECT info -> 'store' -> 'store_name' FROM my_product;
SELECT info ->> 'store' ->> 'store_name' FROM my_product; -- Error

SELECT info -> 'store' ->> 'store_name' FROM my_product;

-- Return the store of all products where the name = 'mango'
SELECT info ->> 'store' FROM my_product WHERE info ->> 'name' = 'mango';
```

## Arrays
- This defines a variable-length multi-dimensional array
- We can create arrays either with the array-literal syntax or ARRAY construct.

```sql
CREATE TABLE test (
    name text,
    items INT ARRAY, -- Array construct. INT ARRAY[2][4]
    square INT[2][4] -- INT[n][m] -> 'n' and 'm' represents the size of each dimension.
);

[   [4,5],
    [5,4,3,5]
] -- [1][1]

--- Insert
-- To insert into an array we can use {} or Array

INSERT INTO test(name, items, square)
VALUES ('kevin', '{2,3,4,5}', ARRAY[[2,3], [5,6]]);

-- one-indexed
-- python list is zero-indexed

SELECT items[1] FROM test;

SELECT items[1] + square[1][1] FROM test;

-- Exercise: Select the first two items in the test table
-- Hint: Use slicing -> [start:stop]
--- postgresql deal just with `start` and `stop` values
-- Since it is one-indexed, the `stop` value is inclusive.
SELECT items[1:2] FROM test;
```